In [1]:
import pandas as pd
from scipy.sparse import csr_matrix, eye, hstack
from sklearn.feature_extraction import DictVectorizer
import numpy as np
from utils.data_processing import train_test_split

Начнём с использования только рейтингов фильма.

In [2]:
data = pd.read_csv('data/rating.csv').iloc[:, :3]

Датасет довольно большой, давайте выберем из него рандомные 2 миллиона строчек. (Немного забегая вперёд: неплохо бы нашей модели уметь давать хорошие рекомендации активным пользователям, потому что тогда есть надежда, что алгоритм будет давать хорошие рекомендации и тем, о ком мы имеем мало данных. Поэтому отбёрём в датасет строчки с самыми активными пользователями.)

In [3]:
active_users = data.userId.value_counts().index - 1
data_part_1 = data.loc[data['userId'].isin(active_users[:5000])]

data_part_2 = data.loc[data['userId'].isin(active_users[:5000]) == False]
rand_users = np.random.choice(data_part_2.shape[0], 
                              size=555000, replace=False)
# Датасет для подбора гиперпараметров
users_for_hyper_tuning = np.random.choice(data_part_2.shape[0], 
                                          size=5000, replace=False)
val_data = data_part_2.iloc[users_for_hyper_tuning, :].sort_values(['userId', 'movieId'])
val_data = val_data.reset_index(drop=True)

data_part_2 = data_part_2.iloc[rand_users, :].sort_values(['userId', 'movieId'])

data = pd.concat([data_part_1, data_part_2], axis=0).sort_values(['userId', 'movieId'])
data = data.reset_index(drop=True)

Добавим две новые колонки в датасет, которые будут отвечать за новые индексы юзеров и фильмов, попавших в data - ***new_userId*** и ***new_movieId*** соответственно.

In [4]:
def modify_dataset(data: pd.DataFrame):
    """
        Args:
            data: pd.DataFrame, данные, в которых идентификаторы пользователей/фильмов
              разбросаны в диапазоне [1, 138493] и [1, 131258] соответственно.
        
        Returns:
            Датафрейм, где добавлены колонки с новыми id для пользователей фильмов в диапазоне
              [1, кол-во уникальных пользователей/фильмов в data].
    """
    user_unique = np.unique(data.userId)
    item_unique = np.unique(data.movieId)
    new_userId = dict([(user_unique[i], i + 1) for i in range(len(user_unique))])
    new_movieId = dict([(item_unique[i], i + 1) for i in range(len(item_unique))])
    col_1 = [int(new_userId[u]) for u in data.userId]
    col_2 = [int(new_movieId[i]) for i in data.movieId]

    new_data = pd.DataFrame(data = {'new_userId': col_1, 'new_movieId': col_2})
    data = pd.concat([new_data, data], axis=1)
    
    return data

In [5]:
data = modify_dataset(data)
data.head()

,new_userId,new_movieId,userId,movieId,rating
0,1,2554,1,2692,3.5
1,1,2860,1,3000,3.5
2,1,5798,1,6093,4.0
3,1,6783,1,7164,3.5
4,1,7642,1,8690,3.5


Создаём user-item матрицу.

In [6]:
def make_interaction(data: pd.DataFrame):
    """
        Args:
            data: pd.DataFrame - данные, содержащие id пользователей и фильмов, 
              а также рейтинги.
        
        Returns:
            User-item матрицу.
    """
    users = np.array(data['new_userId'])
    items = np.array(data['new_movieId'])
    rating = data['rating']
    interaction_sparse = csr_matrix((rating, (users - 1, items - 1)), 
                                    shape=(max(users), max(items)))
    
    return interaction_sparse

In [7]:
interaction_sparse = make_interaction(data)

Разбивать на тренировочный и тестовый набор будем по следующему принципу: хотим уметь хорошо давать рекомендации активным пользователям, так как о них имеем больше информации и скорее всего эти люди заинтересованы продолжать потреблять много контента. Так же, если умеем неплохо давать рекомендации таким пользователям, есть шанс, что алгоритм будет давать неплохие советы и менее активным пользователям, чьё поведение не так хорошо известно.

In [8]:
def get_train_test(data: pd.DataFrame, interaction_sparse: csr_matrix,
                   fraction: float, count: int):
    """
        Args: 
            data: pd.DataFrame - данные для разбиения на тренировочную/тестовую выборки.
            interaction_sparse: csr_matrix - user-item матрица.
            fraction: float - доля пользователей, чьи рейтинги будут рассматриваться
              для занесения в тестовый сет.
            count: int - количество рейтингов, которое будет вноситься в тестовый сет.
        
        Returns:
            train: csr_matrix - тренировочный сет.
            test: csr_matrix - тестовый сет.
            user_ind: id пользователей, чьи оценки были внесены в тестовый сет.
    """
    active_users = data.new_userId.value_counts().index - 1
    train, test, user_ind = train_test_split(interaction_sparse, active_users, 
                                             fraction, count)
    
    return train, test, user_ind

In [9]:
train, test, test_user_ind = get_train_test(data, interaction_sparse, 0.1, 20)

Импортируем метрики, по которым мы будем оценивать алгоритмы.

In [10]:
from utils.metrics import apk, ndcgk

Почему выбраны метрики ***mean average precision at k*** и ***ndcg***? 
Обе метрики позволяют не только вывести самые релевантные объекты пользователю, но и учитывает порядок этих объектов (хочется на первых местах видеть более релевантные фильмы).   
MAP@K хоть и является метрикой для случая, когда рейтинг - бинарная величина, но можно проделать некоторые манипуляции. Поместим текущие оценки в диапазон [0, 1] и установим порог p, после которого фильм с оценкой > p считаем релевантным. Тогда метрика позволит оценить способность алгоритма давать неплохие рекомендации.  
Ndcg же работает и для случая, когда рейтинг - небинарная величина.

Теперь выберем алгоритм.

In [11]:
from lightfm import LightFM

/Applications/anaconda3/lib/python3.9/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


Создадим функцию для подсчёта метрик на тестовом сете.

In [12]:
def calculate_metrics(model: LightFM, test: csr_matrix, item_features,
                      stop: int, k: int, test_user_ind, num_of_items):
    """
        Args:
            model: LightFM - модель, делающая предсказания.
            test: csr_matrix - тестовый сет.
            item_features: None, если не добавляем доп. фичи фильмам, иначе
              это csr_matrix.
            stop: int - величина, которую пришлось добавить из-за длительных расчётов.
              По сути просто прерывает подсчёт метрики на каком-то пользователе из тестового сета.
            k: int - кол-во рекомендаций, которые алгоритм даёт пользователю.
            test_user_ind: идентификаторы пользователей из тестового сета. 
            num_of_items: всего item'ов.
        
        Returns:
            Метрики mean_nDCG@K, MAP@k.
    """
    test_arr = test.toarray()
    ndcgk_per_user = []
    apk_per_user = []
    for i in test_user_ind[:stop]:
        predictions = model.predict(int(i), np.arange(num_of_items), 
                                    item_features=item_features)
        true_val = test_arr[i]
        ndcgk_per_user.append(ndcgk(predictions, true_val, k))
        apk_per_user.append(apk(predictions, true_val, k, 4.0))

    mean_ndcgk = np.sum(ndcgk_per_user)/len(ndcgk_per_user)
    mapk = np.sum(apk_per_user)/len(apk_per_user)
    print('Mean ndcgk:', mean_ndcgk)
    print('Mapk:', mapk)
    
    return mean_ndcgk, mapk

Подберём гиперпараметры на data_val сете. Будем выбирать no_components и learning rate поиском по сетке. В качестве loss'а будем брать warp, так как на практике он себя лучше показывает, чем bpr. Разобьём data_val также на тренировочный и тестовый сет, обучим эпох 30, потом сравним метрики. 

In [13]:
val_data = modify_dataset(val_data)
val_interaction_sparse = make_interaction(val_data)
val_train, val_test, val_user_ind = get_train_test(val_data, val_interaction_sparse, 0.2, 10)

In [14]:
no_components = [20, 30, 40] #Рассматриваю такие значения, потому что при бОльших считается довольно долго, к сожалению.
lr_space = [0.1, 0.05, 0.005]
model_metrics = [{'mean_ndcgk': 0.0, 'mapk': 0.0} for _ in range(9)]
cnt = 0

for n_comp in no_components:
    for lr in lr_space:
        model = LightFM(no_components=n_comp, 
                learning_rate=lr, 
                loss='warp', 
                random_state=7)
        model.fit(val_train, epochs=30)
        print(f'no_components: {n_comp}, learning_rate: {lr}')
        mean_ndcgk, mapk = calculate_metrics(model, val_test, None, len(val_user_ind),
                                             10, val_user_ind, max(val_data['new_movieId']))
        model_metrics[cnt]['mean_ndcgk'] = mean_ndcgk
        model_metrics[cnt]['mapk'] = mapk    
        cnt += 1

no_components: 20, learning_rate: 0.1
Mean ndcgk: 0.011721584189550405
Mapk: 0.0
no_components: 20, learning_rate: 0.05
Mean ndcgk: 0.009849811262948113
Mapk: 0.0
no_components: 20, learning_rate: 0.005
Mean ndcgk: 0.010675659398161548
Mapk: 0.0
no_components: 30, learning_rate: 0.1
Mean ndcgk: 0.012757507005870302
Mapk: 0.0
no_components: 30, learning_rate: 0.05
Mean ndcgk: 0.01247858584119305
Mapk: 0.0
no_components: 30, learning_rate: 0.005
Mean ndcgk: 0.011326454040079937
Mapk: 0.0
no_components: 40, learning_rate: 0.1
Mean ndcgk: 0.010688565909038888
Mapk: 0.0
no_components: 40, learning_rate: 0.05
Mean ndcgk: 0.010261757866642273
Mapk: 0.0
no_components: 40, learning_rate: 0.005
Mean ndcgk: 0.010034184609037846
Mapk: 0.0


Возьмём в качестве гиперпараметров значения:  no_components = 30,  learning_rate = 0.1.

In [15]:
model = LightFM(no_components=30, 
                learning_rate=0.1, 
                loss='warp', 
                random_state=7)

model.fit(train, epochs=50)
# Ограничимся подсчётом метрики для 3000 пользователей из тестового сета, для быстроты вычислений.
mean_ndcgk, mapk = calculate_metrics(model, test, None, 3000, 
                                     10, test_user_ind, max(data['new_movieId']))

Mean ndcgk: 0.07205956776447531
Mapk: 0.0003255291005291005


Теперь импортируем дополнительные сведения о фильме. Добавим в качестве фичей для фильма те теги, которые релевантны хотя бы на 0.25, а также укажем, к какому жанру относится фильм (если вместо жанра стоит (no genres listed), то переопределяем значение на False). Мотивация ясна: фильмы с похожими жанрами и тегами можно будет рекомендовать пользователю, если обнаружится тенденция в его предпочтениях.

In [16]:
data_item_tag_scores = pd.read_csv('data/genome_scores.csv')
data_item_tags = pd.read_csv('data/genome_tags.csv')
# data_item_tags_info - объединение двух предыдущих по общей колонке movieId, чтобы соотнести id тэгов с их названиями
data_item_tags_info = pd.merge(data_item_tags, data_item_tag_scores).iloc[:, 1:]
data_item_tags_info = data_item_tags_info.sort_values('movieId').reset_index(drop=True)
data_genres = pd.read_csv('data/movie.csv').replace('(no genres listed)', value=False)

Создадим фичи для фильмов (добавляем также per-item фичи, чтобы у каждого фильма было закодировано one-hot кодировкой его new_movieId).

In [17]:
def make_item_features(data: pd.DataFrame):
    """
        Args:
            data: pd.DataFrame - данные.
        
        Returns:
            item_features: csr_matrix - матрицу фичей для фильмов. 
    """
    itemId = np.unique(data.new_movieId)
    item_features = [{} for i in itemId]
    for id in itemId:
        corresponding_movieId = data.movieId[data.new_movieId == id].iloc[0]
        info_per_item = data_item_tags_info.loc[data_item_tags_info['movieId'] == corresponding_movieId]
        info_per_item = info_per_item.loc[info_per_item['relevance'] >= 0.25]
        for row in range(info_per_item.shape[0]):
            tag = info_per_item.tag.iat[row]
            relevance = info_per_item.relevance.iat[row]
            item_features[id - 1][f'{tag}'] = relevance

        genres_per_item = data_genres.loc[data_genres['movieId'] == corresponding_movieId]['genres'].iloc[0]
        if genres_per_item:
            for genre in genres_per_item.split('|'):
                item_features[id - 1][genre] = 1
    
    dv = DictVectorizer()
    item_features = dv.fit_transform(item_features)
    eye_matrix = eye(item_features.shape[0], item_features.shape[0]).tocsr()
    item_features = hstack((eye_matrix, item_features)).tocsr().astype(np.float32)
    
    return item_features

In [18]:
item_features = make_item_features(data)

Подберём гиперпараметры для модели с фичами для фильмов. Обучаться будем 20 эпох вследствие несовершенства вычислительных ресурсов.

In [19]:
val_item_features = make_item_features(val_data)
model_with_features_metrics = [{'mean_ndcgk': 0.0, 'mapk': 0.0} for _ in range(9)]
cnt = 0

for n_comp in no_components:
    for lr in lr_space:
        model_with_features = LightFM(no_components=n_comp, 
                learning_rate=lr, 
                loss='warp', 
                random_state=7)
        model_with_features.fit(val_train, item_features=val_item_features, epochs=20)
        print(f'no_components: {n_comp}, learning_rate: {lr}')
        mean_ndcgk, mapk = calculate_metrics(model_with_features, val_test, val_item_features,
                                             len(val_user_ind), 10, val_user_ind, 
                                             max(val_data['new_movieId']))
        model_with_features_metrics[cnt]['mean_ndcgk'] = mean_ndcgk
        model_with_features_metrics[cnt]['mapk'] = mapk    
        cnt += 1

no_components: 20, learning_rate: 0.1
Mean ndcgk: 0.0020866461283098087
Mapk: 0.0
no_components: 20, learning_rate: 0.05
Mean ndcgk: 0.0022076942197074936
Mapk: 0.0
no_components: 20, learning_rate: 0.005
Mean ndcgk: 0.003670895598052078
Mapk: 0.0
no_components: 30, learning_rate: 0.1
Mean ndcgk: 0.0003585036904467763
Mapk: 0.0
no_components: 30, learning_rate: 0.05
Mean ndcgk: 0.00022029655131030704
Mapk: 0.0
no_components: 30, learning_rate: 0.005
Mean ndcgk: 0.0011589004193791168
Mapk: 0.0
no_components: 40, learning_rate: 0.1
Mean ndcgk: 0.0004576796578888343
Mapk: 0.0
no_components: 40, learning_rate: 0.05
Mean ndcgk: 0.004514112352265212
Mapk: 0.0
no_components: 40, learning_rate: 0.005
Mean ndcgk: 0.00333057386039487
Mapk: 0.0


Обучим модель со следующими гиперпараметрами: no_components = 40,  learning_rate = 0.05.

In [21]:
model_with_features = LightFM(no_components=40, 
                learning_rate=0.05, 
                loss='warp', 
                random_state=7)

model_with_features.fit(train, item_features=item_features, epochs=50)
mean_ndcgk, mapk = calculate_metrics(model_with_features, test, item_features,
                                     3000, 10, test_user_ind, max(data['new_movieId']))

Mean ndcgk: 0.05526614095399426
Mapk: 0.5767131029464363


Теперь сравним с бейзлайном (просто выдавать самые популярные фильмы).

In [22]:
def predictions_k(data: pd.DataFrame, k: int):
    """
        Args:
            data: pd.DataFrame - данные.
            k: int - кол-во рекомендаций, которые алгоритм даёт пользователю.
            
        Returns:
            Рекомендации, составленные из самых популярных фильмов с рейтингом >= 4.0.
    """
    popular_films = data.new_movieId.loc[data.rating >= 4.0].value_counts().index - 1
    n_films = len(np.unique(data.new_movieId))
    pred = np.zeros(shape = n_films)
    pred[popular_films[:k]] = 5.0
    
    return pred

In [23]:
test_arr = test.toarray()
ndcgk_per_user_base = []
apk_per_user_base = []
predictions = predictions_k(data, 10)
for i in test_user_ind[:3000]:
    true_val = test_arr[i]
    ndcgk_per_user_base.append(ndcgk(predictions, true_val, 10))
    apk_per_user_base.append(apk(predictions, true_val, 10, 4.0))

mean_ndcgk = np.sum(ndcgk_per_user_base)/len(ndcgk_per_user_base)
mapk = np.sum(apk_per_user_base)/len(apk_per_user_base)
print('Mean ndcgk:', mean_ndcgk)
print('Mapk:', mapk)

Mean ndcgk: 0.11852914487178769
Mapk: 0.05323333333333333


***Вывод*** 

С метрикой ***mean_nDCG@K*** происходит что-то странное, раз на baseline'е она выдаёт лучше результат, чем на обученных моделях, возможно проблемы с реализацией, которые не удалось отследить. 

А вот судя по ***MAP@K*** модель с фичами лучше даёт рекомендации пользователям. Изначально было предположение, что из-за наличия доп. информации о фильмах модель может переобучиться и соответственно хуже выдавать фильмы. Но, видимо, тщательный отбор признаков помог уловить тенденции в предпочтениях пользователей. 

Хотелось бы конечно обучить модели на всех имеющихся данных, тогда возможно и простая модель с рейтингами показала бы лучше результаты, но такие вычисления, к сожалению, очень затратно производить на имеющемся оборудовании. Также не исключено, что даже чуть больший выбор гиперпараметров дал бы лучше результаты (была попытка обучить модель с фичами, у которых no_components = 60, но пришлось прервать вычисления из-за длительности вычислений). 